In [ ]:
import polars as pl
import polars_ds as pds
print(pds.__version__)

In [ ]:
size = 1000
df = pds.frame(size = size).select(
    pds.random(0., 1.).alias("x1"),
    pds.random(0., 1.).alias("x2"),
    pds.random(0., 1.).alias("x3"),
).with_columns(
    pl.when(pds.random() < 0.2).then(None).otherwise(pl.col("x1")).alias("x1"),
    pl.when(pds.random() < 0.2).then(None).otherwise(pl.col("x2")).alias("x2"),
    pl.when(pds.random() < 0.2).then(None).otherwise(pl.col("x3")).alias("x3"),
).with_columns(
    null_ref = pl.any_horizontal(pl.col("x1").is_null(), pl.col("x2").is_null(), pl.col("x3").is_null()),
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)

window_size = 6
min_valid_rows = 5

result = df.with_columns(
    pds.query_rolling_lstsq(
        "x1", "exp(x2)", "x3",
        target = "y",
        window_size = window_size,
        min_valid_rows = min_valid_rows,
        null_policy = "skip" 
    ).alias("test")
).with_columns(
    pl.col("test").is_null().alias("is_null")
)

In [ ]:
pl.sql_expr(pl.col("x1"))

In [ ]:
result.head(10)

In [ ]:
df.with_columns(
    pds.query_rolling_lstsq(
        "x1", "x2", "x3",
        target = "y",
        window_size = 5,
        min_valid_rows = 4,
        null_policy = "skip" 
    ).alias("test")
) # .unnest("test").limit(10)

In [ ]:
df = pl.DataFrame({
    "x1": [None, 0.1, 0.1, 0.1],
    "x2": [None, 0.1, 0.1, 0.1],
    "x3": [None, 0.1, 0.1, 0.1],
})

In [ ]:
df.slice(offset=1).with_columns(
    pds.query_recursive_lstsq(
        "x1", "x2", "x3",
        target = "y",
        start_with = 2,
        null_policy = "zero" # fill with zero
    ).alias("test")
).unnest("test").limit(10)

In [ ]:
df.select(
    pds.query_knn_ptwise(
        "x1", "x2", "x3",
        index = "index",
        dist = "l2",
        k = 2
    ).over("category")
)

In [ ]:
df.with_columns(
    pds.query_recursive_lstsq(
        "x1", "x2", "x3",
        target = "y",
        start_at = 3,
        method = "normal",
    ).alias("res")
).unnest("res")

In [ ]:
df.with_columns(
    pds.query_rolling_lstsq(
        "x1", "x2", "x3",
        target = "y",
        window_size = 60,
        method = "l2",
        # add_bias = True,
        lambda_ = 0.01
    ).alias("res")
).unnest("res")